# Explore signal/price movement relationship

This is an example notebook to explore whether a trading signal results to profitable trades.

- 3 days 
- It explores a small set of pairs on Uni v3 on Polygon
- Both long and short are considered (though shorts might be theoretical only, if such a lending market doest not exist in the point of time)

First run to prepare the dataset:

```shell
python scripts/prepare-polygon-momentum-candles.py
```

In [ ]:
import pandas as pd

from tradingstrategy.client import Client
from tradingstrategy.chain import ChainId
from tradingstrategy.pair import PandasPairUniverse
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.execution_context import notebook_execution_context
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.strategy_module import read_strategy_module
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, load_partial_data, load_trading_and_lending_data
from tradeexecutor.strategy.universe_model import UniverseOptions

client = Client.create_jupyter_client()



### Interactive viewer mode

Set Plotly chart output mode to interactive viewing.

In [ ]:
from tradeexecutor.utils.notebook import OutputMode, setup_charting_and_output

# setup_charting_and_output(OutputMode.interactive, max_rows=35, height=1000)
setup_charting_and_output(
    OutputMode.static, 
    image_format="png",
    width=1500,
    height=1200,
    max_rows=30,    
    #min_rows=10,
)

#pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.float_format = '{:,.2f}'.format


## Filter trading pairs

In [ ]:
from tradingstrategy.pair import filter_for_stablecoins, StablecoinFilteringMode

candles_df = all_candles_df

print(f"We have {len(pair_ids)} tradeable pairs")
print(f"We have {len(candles_df):,} candles for tradeable pairs, out of {len(all_candles_df):,} total candles")


## Explore signal vs price change

Create a function `calculate_signal_vs_profit` which calculates 
- Signal (naive momentum)
- Profit: the last and the best future price we can get
- Allows us to play around with different time windows
- Split between shorts and longs


In [ ]:
from pandas.tseries.frequencies import to_offset
import humanize

# Set up parameters of time windows
# we use for this notebook run
#lookback_window = pd.Timedelta(days=2)
#profit_window = pd.Timedelta(days=2)

lookback_window = pd.Timedelta(hours=4)
profit_window = pd.Timedelta(hours=4)

long_lookback_window = lookback_window * 4
daily_volume_threshold = 10_000

# short - long EMA pairs
short_long_ema_pairs = [
    (2, 4),
    # (16, 48),
    # (32, 96),
]

# Normalise EMA diff to 90 days/90 hours
ema_diff_short_normalisation_period = 24*7
ema_diff_long_normalisation_period =  30*24

min_entries = 14

min_age = pd.DateOffset(days=20)

zero_signal_cut_off = 0.0001

quantile_sanity_threshold = 0.9995

#signal_source = "momentum"
signal_source = "weighted_ema"

apply_filter_by_signal = False
apply_filter_by_profit = False
apply_response_function = False

# Use later in chart titles
signal_window_label = humanize.naturaldelta(lookback_window)
profit_window_label = humanize.naturaldelta(profit_window)


Create a function `calculate_signal_vs_price_for_pair` which calculates 
- Calculates the signal vs. for certain trading pair
- Bundle few different pairs to the same `DataFrame` so we can examine them together


## Grid search function

In [ ]:
import importlib
import sys

grid_search_mod_path = get_default_strategies_path() / "grid-search" / "binance-sma.py"

# See https://stackoverflow.com/a/67692/315168
spec = importlib.util.spec_from_file_location("binance-sma", grid_search_mod_path)
python_mod = importlib.util.module_from_spec(spec)
sys.modules["binance-sma"] = python_mod
spec.loader.exec_module(python_mod)

print(f"Grid searching for for strategy {grid_search_mod_path}")

## Execute grid search

In [ ]:
import runpy

mod = runpy.run_path(grid_search_mod_path)
mod["main"]()

## Show grid search results

In [ ]:
display(result_df)